<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Google-Cloud-Speech-to-Text-API-Microphone-Streaming-emulate" data-toc-modified-id="Google-Cloud-Speech-to-Text-API-Microphone-Streaming-emulate-1">Google Cloud Speech-to-Text API Microphone Streaming emulate</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Microphone-Streaming-emulate" data-toc-modified-id="Microphone-Streaming-emulate-1.0.1">Microphone Streaming emulate</a></span></li><li><span><a href="#위의-기능을-Python-module로-작성한-기능-테스트" data-toc-modified-id="위의-기능을-Python-module로-작성한-기능-테스트-1.0.2">위의 기능을 Python module로 작성한 기능 테스트</a></span></li></ul></li></ul></li></ul></div>

# Google Cloud Speech-to-Text API Microphone Streaming emulate

- Microphone 오디오 파일을 Streaming 방식으로 처리하는 샘플(실제는 저장된 오디오 파일을 읽어서 Streaming 방식으로 처리)
- https://cloud.google.com/speech-to-text/docs/streaming-recognize?hl=ko#speech-streaming-mic-recognize-python
- **Microphone Streaming 오디오 처리는 300초 시간 제한이 있음** : 더 긴 파일은 파일을 잘라서 처리하는 방법을 사용할 수 있음<br>
   (OutOfRange: 400 Exceeded maximum allowed stream duration of 305 seconds.)

###  Microphone Streaming emulate

In [1]:
from __future__ import division

import sys
import re
import wave

from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

import pyaudio
from six.moves import queue

# Audio recording parameters
RATE = 16000
CHUNK = int(RATE / 10)  # 100ms

In [2]:
class MicrophoneStream(object):
    """Opens a recording stream as a generator yielding the audio chunks."""
    def __init__(self, rate, chunk):
        self._rate = rate
        self._chunk = chunk

        # Create a thread-safe buffer of audio data
        self._buff = queue.Queue()
        self.closed = True

    def __enter__(self):
        self._audio_interface = pyaudio.PyAudio()
        self._audio_stream = self._audio_interface.open(
            format=pyaudio.paInt16,
            # The API currently only supports 1-channel (mono) audio
            # https://goo.gl/z757pE
            channels=1, rate=self._rate,
            input=True, frames_per_buffer=self._chunk,
            # Run the audio stream asynchronously to fill the buffer object.
            # This is necessary so that the input device's buffer doesn't
            # overflow while the calling thread makes network requests, etc.
            stream_callback=self._fill_buffer,
        )

        self.closed = False

        return self

    def __exit__(self, type, value, traceback):
        self._audio_stream.stop_stream()
        self._audio_stream.close()
        self.closed = True
        # Signal the generator to terminate so that the client's
        # streaming_recognize method will not block the process termination.
        self._buff.put(None)
        self._audio_interface.terminate()

    def _fill_buffer(self, in_data, frame_count, time_info, status_flags):
        """Continuously collect data from the audio stream, into the buffer."""
#         self._buff.put(in_data)
        data = wf.readframes(frame_count)
        self._buff.put(data)
        stream.write(data)
        return None, pyaudio.paContinue

    def generator(self):
        while not self.closed:
            # Use a blocking get() to ensure there's at least one chunk of
            # data, and stop iteration if the chunk is None, indicating the
            # end of the audio stream.
            chunk = self._buff.get()
            if chunk is None:
                return
            data = [chunk]

            # Now consume whatever other data's still buffered.
            while True:
                try:
                    chunk = self._buff.get(block=False)
                    if chunk is None:
                        return
                    data.append(chunk)
                except queue.Empty:
                    break

            yield b''.join(data)

In [3]:
def listen_print_loop(responses):
    """Iterates through server responses and prints them.

    The responses passed is a generator that will block until a response
    is provided by the server.

    Each response may contain multiple results, and each result may contain
    multiple alternatives; for details, see https://goo.gl/tjCPAU.  Here we
    print only the transcription for the top alternative of the top result.

    In this case, responses are provided for interim results as well. If the
    response is an interim one, print a line feed at the end of it, to allow
    the next result to overwrite it, until the response is a final one. For the
    final one, print a newline to preserve the finalized transcription.
    """
    num_chars_printed = 0
    for response in responses:
        if not response.results:
            continue

        # The `results` list is consecutive. For streaming, we only care about
        # the first result being considered, since once it's `is_final`, it
        # moves on to considering the next utterance.
        result = response.results[0]
        if not result.alternatives:
            continue

        # Display the transcription of the top alternative.
        transcript = result.alternatives[0].transcript

        # Display interim results, but with a carriage return at the end of the
        # line, so subsequent lines will overwrite them.
        #
        # If the previous result was longer than this one, we need to print
        # some extra spaces to overwrite the previous result
        overwrite_chars = ' ' * (num_chars_printed - len(transcript))

        if not result.is_final:
            sys.stdout.write(transcript + overwrite_chars + '\r')
            sys.stdout.flush()

            num_chars_printed = len(transcript)

        else:
            print(transcript + overwrite_chars)

            # Exit recognition if any of the transcribed phrases could be
            # one of our keywords.
            if re.search(r'\b(exit|quit)\b', transcript, re.I):
                print('Exiting..')
                break

            num_chars_printed = 0

In [4]:
def microphone_streaming_start():
    # See http://g.co/cloud/speech/docs/languages
    # for a list of supported languages.
    language_code = 'ko-KR'

    client = speech.SpeechClient()
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=RATE,
        language_code=language_code,
        enable_automatic_punctuation=True,
        enable_word_time_offsets=True)
    #     enable_speaker_diarization=True,
    #     diarization_speaker_count=3)

    streaming_config = types.StreamingRecognitionConfig(
        config=config,
        interim_results=True)

    with MicrophoneStream(RATE, CHUNK) as stream:
        audio_generator = stream.generator()
        requests = (types.StreamingRecognizeRequest(audio_content=content)
                    for content in audio_generator)

        responses = client.streaming_recognize(streaming_config, requests)

        # Now, put the transcription responses to use.
        listen_print_loop(responses)

In [5]:
FILE_NAME = "19m_20181220_deominju_16000.wav"
wf = wave.open("audio_file/"+ FILE_NAME, 'rb')     # 300초 이상의 파일은 분할해서 처리 해야 함

p = pyaudio.PyAudio()

stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                channels=wf.getnchannels(),
                rate=wf.getframerate(),
                output=True)

In [6]:
# 중단된 경우 재실행 하면 중단된 부분부터 시작됨
try:
    microphone_streaming_start()
except KeyboardInterrupt:
    pass

In [7]:
stream.stop_stream()
stream.close()
p.terminate()

### 위의 기능을 Python module로 작성한 기능 테스트

In [1]:
from modules import google_stt as stt

In [2]:
import wave
import pyaudio

In [3]:
FILE_NAME = "witch_hunt_2_online-audio-converter.com_16000.wav"
wf = wave.open("audio_file/"+ FILE_NAME, 'rb')     # 300초 이상의 파일은 분할해서 처리 해야 함

p = pyaudio.PyAudio()

stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                channels=wf.getnchannels(),
                rate=wf.getframerate(),
                output=True)

In [4]:
try:
    stt.microphone_streaming_start(wf, stream)    # 함수 호출을 위해 객체 전달
except KeyboardInterrupt:
    pass
except Exception:
    pass

근데 연예인 중에서도 함께 많이 방송을 뭘 하게 될 일이 생기면 전전긍긍하고 제작장 댁에서 좀 빼 달라 나는 같이 지금 할 수가 끝까지 그런 거 조심하는 경우가 있고 어떤 분들은 또 천연덕스럽게 조는 괜찮고 훌륭하게 대처하는 분들도 계시더라구요. 어때요? 방송에 나와요. 뭐 예전에 잠깐 이렇게 사겼던 그런 사람인데 갑자기 반천계곡 예. 예 많이 힘들죠. 그냥 조그만 맞는구나 해서 헤어진 가벼운 관계는 그래. 우리 이모 친구는 계속 지내자. 근데 정말 막 축구 못살고 막 그러다가 어쩔 수 없는 이유로 헤어졌으면 나중에 만나서 친구 잘 지내는가? 이렇게 되는 거 이상한 거 아닌가요? 잘 지내는가?  
좋아했던 사람 자료 왔는가 보통 치료가 잘 안 되는 거예요. 그냥 쉽게 쉽게 만나는 스타일이 수도 있을 것 같아요. 근데 문제는이 여자가 딴 거를 못 해. 그렇지. 이런데 사 준다는 거야이 말을 할 때 얼마나 많은게 맞는 얘기고 못하는 얘기하며 방송용으로 창동역 역삼동 헤어졌어요. 상대방에게 다시 연락 없어요. 미친 거 같아. 다음에 술 마시고 전화 한 거야. 그래서 막 약속할 수 없는 말들을 자꾸 문자로 보내는 거예요. 다음날 되면 또 필요하거든. 당연히 병원에 있어요. 좀 받아 본 적이 있죠 있어요. 지금 어디야? 왜 보고 싶어서
여보세요. 마이크가 어디야? 뉘앙스가 그냥 지금 만나서 오늘 같이 있자. 뭐 이런 느낌이었어요. 아니, 그렇게 만나서 회포를 푸순 있겠죠? 하룻밤 근데 없고 하루 지나고 난 술도 깨고 나면 사랑하지 못하는 그런게 있지 않냐? 술취해서 전화한 거 그다음에 일어나는데 발신 열 두 번인데 0초 0초 0초 0초 0초 12분 이런 거 있잖아요. 참 술 먹었을 때 아침 일어나지마 주종 했던 거 핸드폰 확인했어요. 했나 누구한테 전화했는지 누구한테 문자 보냈는지 그리고 무슨 사진 찍은지
아니면 큰 나라는 기억안날때 무슨 사진 있는지 아니? 근데 이렇게 보고 소스라치게 놀란 적이 있지요. 이런 사진 왜 그런 거였지? 어? 거기가 있었지? 그렇군요. 지

In [6]:
stream.stop_stream()
stream.close()
p.terminate()